# Grade Prediction

In [2]:
# Get dataset
import kagglehub
import os

path = kagglehub.dataset_download("samira1992/student-grades-intermediate-dataset")

for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        path = os.path.join(dirname, filename)

path

/home/jm/Documents/code/git/Machine_Learning/Projects/Grade_Prediction/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/home/jm/.cache/kagglehub/datasets/samira1992/student-grades-intermediate-dataset/versions/1/Student_Grades.csv'

In [3]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
import torchvision.transforms.v2 as transforms

%matplotlib inline

dataframe = pd.read_csv(path)

dataframe.head(5)

,Hours,Practice,TeamWork,MidTerm,FinalExam,Scores,Grade
0,2.5,0.5,0.0,2,2.3,21,C
1,5.1,1.5,0.5,4,4.9,47,B
2,3.2,1.0,0.5,3,2.9,27,C
3,8.5,3.7,1.2,7,7.7,75,A
4,3.5,1.2,0.3,3,3.2,30,C


In [4]:
# Mean Squared Error
def MSE(prediction, target):
    return torch.mean(prediction - target) ** 2
# Split features in dataset
y = dataframe["Scores"]
x = dataframe.drop(columns=["Scores", "Grade"])

In [8]:
x.head(5)

,Hours,Practice,TeamWork,MidTerm,FinalExam
0,2.5,0.5,0.0,2,2.3
1,5.1,1.5,0.5,4,4.9
2,3.2,1.0,0.5,3,2.9
3,8.5,3.7,1.2,7,7.7
4,3.5,1.2,0.3,3,3.2


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# Create base model
base = torch.nn.Sequential(
    torch.nn.Linear(5, 16),
    torch.nn.ReLU(),
    torch.nn.Linear(16, 32),
    torch.nn.ReLU(),
    torch.nn.Linear(32, 1)
)
# Converts data to float32 tensor
preprocess_trans = transforms.Compose([
    transforms.ToDtype(torch.float32)
])

loss_function = torch.nn.MSELoss()
optimizer = Adam(base.parameters())
# Compile to gpu
model = torch.compile(base.to(device))

In [ ]:
def train():
    batch_size = 16
    train_loader = DataLoader(x, batch_size, True)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data) -> None:
        super().__init__()